# GPT2 as text classifier

Helpful source: https://drlee.io/fine-tuning-gpt-2-for-sentiment-analysis-94ebdd7b5b24

## Install Required Libraries


In [1]:
!pip install datasets # unified interface for accessing and working with various datasets (by hugging face)
!pip install -U accelerate # library to optimize and accelerate numerical computations
!pip install -U transformers # library by hugging face that gives easy access to pre-trained models, tokenizers, and tools for fine-tuning models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

## Loading and Processing the Dataset

We load the dataset from Hugging Face. Each sample consists of one strings feature that stores the title as well as the (start of the) article-text. The label is the category that the article belongs to (world, sports, business, sci/tech). [Link](https://huggingface.co/datasets/ag_news/viewer/default/train) to explore the structure of the data.

In [2]:
from datasets import load_dataset

dataset = load_dataset('ag_news')

print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


Reduce the size of the dataset (to reduce training times) whilst ensuring that the original structure and distribution of the data is kept.

In [3]:
from datasets import Dataset, DatasetDict
import pandas as pd

def take_a_percentage_of_data(dataset, percentage=0.1, shuffle=True, random_state=None):
    # sort and group the dataset by label
    df = pd.DataFrame(dataset)
    df_sorted = df.sort_values(by='label')
    grouped_dfs = df_sorted.groupby('label')

    # ensure that proportions of the groups remains the same as in the original dataset
    filtered_dfs_per_group = []
    for label, group in grouped_dfs:
        num_samples_to_keep = int(len(group) * percentage)
        filtered_group = group.head(num_samples_to_keep)
        filtered_dfs_per_group.append(filtered_group)

    # concatenate (and shuffle) the filtered group-wise dataframes
    filtered_df = pd.concat(filtered_dfs_per_group)
    if shuffle:
        filtered_df = filtered_df.sample(frac=1, random_state=random_state)

    filtered_df.reset_index(drop=True, inplace=True) # resets the index of the DataFrame, drops the previous index column
    filtered_df_as_dict = filtered_df.to_dict(orient='list')
    filtered_dataset = Dataset.from_dict(filtered_df_as_dict)
    return filtered_dataset

dataset_train_1percent = take_a_percentage_of_data(dataset['train'], percentage=0.01)
dataset_test_1percent = take_a_percentage_of_data(dataset['test'], percentage=0.01)

dataset_1percent = DatasetDict({
    'train': dataset_train_1percent,
    'test': dataset_test_1percent
}) # combine the shortened datasets back into the old structure.

print(dataset_1percent)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 76
    })
})


## Tokenizing the dataset

Tokenize the dataset in the exact same way as the GPT-2 model.

In [4]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token # padding tokens added to sequences will be represented by an end-of-sequence token
def tokenize_function(examples):
    #return tokenizer(examples["text"], padding="max_length", truncation=True)
    return tokenizer(examples["text"], padding="max_length")

tokenized_dataset = dataset_1percent.map(tokenize_function, batched=True) # performed in batches to increase performance

print(tokenized_dataset) # tokenization adds two features: 'input_ids' (the tokenized representation of 'text') as well as 'attention_mask', which ensures that the model does not attend to padding tokens added during tokenization

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 76
    })
})


## Loading the Pre-trained GPT2-Model

Load the pre-trained GPT2-Model for sequence classification.

In [5]:
from transformers import GPT2ForSequenceClassification

gpt2_model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=4) # our gpt2-model should distinguish between 4 labels, adds a final fully connected layers with 4 output neurons.

print(gpt2_model)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


## Implementing the Laplacian Adaptive Attention Block

The code below was extracted from the paper and then adapted to use a Laplacian probability distribution rather than Gaussian probability distribution.

In [6]:
import torch
import torch.nn as nn

class LaplacianAdaptiveAttention(nn.Module):
    def __init__(self, norm_axis, num_heads, num_laplacians, padding_value, mean_offset_init=0, eps=1e-8):
        super().__init__()
        if not isinstance(norm_axis, int):
            raise ValueError("norm_axis must be an integer.")
        if num_heads <= 0 or not isinstance(num_heads, int):
            raise ValueError("num_heads must be a positive integer.")
        if num_laplacians <= 0 or not isinstance(num_laplacians, int):
            raise ValueError("num_laplacians must be a positive integer.")

        self.norm_axis = norm_axis
        self.eps = eps
        self.num_heads = num_heads
        self.padding_value = padding_value
        self.num_laplacians = num_laplacians

        self.mean_offsets = nn.Parameter(torch.zeros(num_laplacians, dtype=torch.float)) # offsets are initialized with 0.
        self.c = nn.Parameter(torch.randn(num_laplacians, dtype=torch.float))

    def forward(self, x, return_attention_details=False):
        if x.dim() < 2:
            raise ValueError(f"Input tensor must have at least 2 dimensions, got {x.dim()}.")
        if self.norm_axis >= x.dim() or self.norm_axis < -x.dim():
            raise ValueError(f"norm_axis {self.norm_axis} is out of bounds for input tensor with {x.dim()} dimensions.")

        mask = x != self.padding_value if self.padding_value is not None else None
        x_masked = torch.where(mask, x, torch.zeros_like(x)) if mask is not None else x
        median = x_masked.median(dim=self.norm_axis, keepdim=True)[0] # ADAPTED
        b = torch.abs(x_masked - median).mean(dim=self.norm_axis, keepdim=True) + self.eps # ADAPTED

        mixture = 1
        for i in range(self.num_laplacians):
            adjusted_median = median + self.mean_offsets[i] # ADAPTED (names)
            y_norm = (x - adjusted_median) / torch.sqrt(b) # ADAPTED (names) - how does normalization occur for Laplacian distributions?
            laplacian = torch.exp(-(torch.abs(y_norm) / (2.0 * (self.c[i] ** 2)))) / torch.sqrt(2 * torch.pi * (self.c[i] ** 2)) # ADAPTED - equation (9), but second division term cannot be found in the paper - likely the scaling term.
            mixture *= laplacian

        mixture /= mixture.sum(dim=self.norm_axis, keepdim=True).clamp(min=self.eps)

        if return_attention_details:
            return torch.where(mask, x * mixture, x) if mask is not None else x * mixture, mixture.detach()
        else:
            return torch.where(mask, x * mixture, x) if mask is not None else x * mixture


class MultiHeadLaplacianAdaptiveAttention(nn.Module):
    def __init__(self, norm_axis, num_heads, num_laplacians, padding_value=None, eps=1e-8):
        super().__init__()
        self.norm_axis = norm_axis
        self.num_heads = num_heads
        self.attention_heads = nn.ModuleList([
            LaplacianAdaptiveAttention(norm_axis, num_heads, num_laplacians, padding_value, eps)
            for _ in range(num_heads)
        ])

    def forward(self, x, return_attention_details=False):
        chunk_size = x.shape[self.norm_axis] // self.num_heads
        if chunk_size == 0:
            raise ValueError(f"Input tensor size along norm_axis ({self.norm_axis}) must be larger than the number of heads ({self.num_heads}).")

        outputs, attention_details_ = [], []
        for i in range(self.num_heads):
            start_index = i * chunk_size
            end_index = start_index + chunk_size if i < self.num_heads - 1 else x.shape[self.norm_axis]
            chunk = x.narrow(self.norm_axis, start_index, end_index - start_index)
            if return_attention_details:
                out, mixture = self.attention_heads[i](chunk, return_attention_details=True)
                outputs.append(out)
                attention_details_.append(mixture)
            else:
                outputs.append(self.attention_heads[i](chunk))

        if return_attention_details:
            return torch.cat(outputs, dim=self.norm_axis), torch.cat(attention_details_, dim=self.norm_axis)
        else:
            return torch.cat(outputs, dim=self.norm_axis)


class LaplacianBlock(nn.Module):
    def __init__(self, norm_axes, num_heads, num_laplacians, num_layers, padding_value=None, eps=1e-8):
        super().__init__()
        if len(norm_axes) != num_layers or len(num_heads) != num_layers or len(num_laplacians) != num_layers:
            raise ValueError("Lengths of norm_axes, num_heads, and num_laplacians must match num_layers.")

        self.layers = nn.ModuleList([
            MultiHeadLaplacianAdaptiveAttention(norm_axes[i], num_heads[i], num_laplacians[i], padding_value, eps)
            for i in range(num_layers)
        ])

    def forward(self, x, return_attention_details=False):
        attention_details_ = {}
        for idx, layer in enumerate(self.layers):
            if return_attention_details:
                x_, attention_details = layer(x, return_attention_details=True)
                attention_details_['layer_'+str(idx)] = attention_details
                x = x_ + x
            else:
                x = layer(x) + x

        if return_attention_details:
            return x, attention_details_
        return x

**Approach**: Create a wrapper class of MultiHeadLaplacianAdaptive Attention that is then inserted into the GPT2-architecture layer by layer.

In [7]:
import importlib
import torch
import copy

laplacian_model = copy.deepcopy(gpt2_model)

class MultiHeadLaplacianAdaptiveAttentionWrapper(torch.nn.Module):
    def __init__(self, config, num_laplacians=10, norm_axis=1):
        super().__init__()
        self.attention = MultiHeadLaplacianAdaptiveAttention(
            norm_axis=norm_axis, # same as example of the researchers (dario had = 2)
            num_heads=config.n_head,
            num_laplacians=num_laplacians,
            padding_value=config.eos_token_id,
            eps=config.layer_norm_epsilon
        )

    def forward(self, hidden_states, **kwargs):
        # Pass arguments using **kwargs to the underlying attention mechanism
        attention_output = self.attention(hidden_states)
        return (hidden_states,) + tuple(attention_output)  # Ensure the return value is a tuple


# Replace the attention mechanism in each transformer block
for block in laplacian_model.transformer.h: # accessing each transformer blocks within the GPT-2 model
    block.attn = MultiHeadLaplacianAdaptiveAttentionWrapper(config=laplacian_model.config, num_laplacians=5, norm_axis=1) # and replacing the attention module with the Gaussian attention block.


In [8]:
print(laplacian_model)
print(gpt2_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadLaplacianAdaptiveAttentionWrapper(
          (attention): MultiHeadLaplacianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-11): 12 x LaplacianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
  

## Freezing Parameters

For the first training epochs, we decided to freeze the pre-trained layers. We did this, because we want the attention weights to first learn some general things before we fine-tune the model as a whole.

In [9]:
for param in laplacian_model.transformer.parameters():
    param.requires_grad = False

for param in laplacian_model.score.parameters():
    param.requires_grad = True

for block in laplacian_model.transformer.h:
  for param in block.attn.parameters():
      param.requires_grad = True


In [10]:
trainable_params = sum(p.numel() for p in laplacian_model.parameters() if p.requires_grad)
frozen_params = sum(p.numel() for p in laplacian_model.parameters() if not p.requires_grad)

print(f"Trainable Parameters: {trainable_params}")
print(f"Frozen Parameters: {frozen_params}")

Trainable Parameters: 4512
Frozen Parameters: 96091392


## Check: Do both models run?

1. The Normal GPT2-Model
2. The GPT2-Model with LAAM

In [11]:
input_ids = torch.randint(0, gpt2_model.config.vocab_size, (1, 512))
labels = torch.tensor([1]).unsqueeze(0)

outputs = gpt2_model(input_ids=input_ids, labels=labels)
loss, logits = outputs['loss'], outputs['logits']
print(f"GPT2 with regular attention mechanism: Loss = {round(loss.item(), 4)}, logits = {logits.detach()}")

outputs = laplacian_model(input_ids=input_ids, labels=labels)
loss, logits = outputs['loss'], outputs['logits']
print(f"GPT2 with Laplacian attention mechanism: Loss = {round(loss.item(), 4)}, logits = {logits.detach()}")


GPT2 with regular attention mechanism: Loss = 10.3276, logits = tensor([[ 1.9818, -6.3446,  3.4121,  2.7781]])
GPT2 with Laplacian attention mechanism: Loss = 5.679, logits = tensor([[ 1.1515, -2.3336,  2.6956,  2.3320]])


## Training

Training the GPT2-Model with Laplacian attention

In [12]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    # weight_decay=0.01,
    use_cpu = False,
    no_cuda = False
)

trainer = Trainer(
    model=laplacian_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


In [13]:
model_name = "gpt2-ag_news-1percent-frozen-laplacian"

trainer.train()
trainer.save_model(model_name)
# Save the tokenizer used by the model as well
tokenizer.save_pretrained(model_name)


Step,Training Loss
500,1.532100
1000,1.517000
1500,1.490700
2000,1.495500
2500,1.454300
3000,1.484500
3500,1.476000
4000,1.443900
4500,1.435400
5000,1.431300


('gpt2-ag_news-1percent-frozen-laplacian/tokenizer_config.json',
 'gpt2-ag_news-1percent-frozen-laplacian/special_tokens_map.json',
 'gpt2-ag_news-1percent-frozen-laplacian/vocab.json',
 'gpt2-ag_news-1percent-frozen-laplacian/merges.txt',
 'gpt2-ag_news-1percent-frozen-laplacian/added_tokens.json')

In [14]:
trainer.evaluate()

{'eval_loss': 1.3873882293701172,
 'eval_runtime': 14.4488,
 'eval_samples_per_second': 5.26,
 'eval_steps_per_second': 5.26,
 'epoch': 5.0}

## Calculating the accuracy

In [15]:
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

laplacian_model.to(device)

def predict_label(sentence, model):
    inputs = tokenizer(sentence, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        prediction = outputs.logits.argmax(-1).item()

    return prediction

def calculate_accuracy(model, data):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for item in data:
            prediction = predict_label(item['text'], model)
            label = item['label']
            total += 1
            if label == prediction:
              correct += 1

    return correct / total

data = dataset_1percent['test']

# Calculate accuracy
accuracy = calculate_accuracy(laplacian_model, data)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.2368


## Second Training Cycle with Unfrozen Layers

Loading the previously trained model and unfreezing the layers.

In [16]:
unfrozen_laplacian_model = copy.deepcopy(laplacian_model)
print(unfrozen_laplacian_model)

# unfreeze layers
for param in unfrozen_laplacian_model.transformer.parameters():
    param.requires_grad = True

# check
trainable_params = sum(p.numel() for p in unfrozen_laplacian_model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params}")

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadLaplacianAdaptiveAttentionWrapper(
          (attention): MultiHeadLaplacianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-11): 12 x LaplacianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)
Trainable Parameters: 96095904


Defining the new parameters for the second training cycle (smaller learning rate, including weight decay)

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    weight_decay=0.01,
    use_cpu = False,
    no_cuda = False
)

trainer = Trainer(
    model=unfrozen_laplacian_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


Starting the second training cycle.

In [18]:
trainer.train()

Step,Training Loss
500,1.387500
1000,1.384400
1500,1.386600
2000,1.379100
2500,1.380100
3000,1.390200
3500,1.394400
4000,1.385700
4500,1.383900
5000,1.394700


TrainOutput(global_step=6000, training_loss=1.3872383524576823, metrics={'train_runtime': 2426.2887, 'train_samples_per_second': 2.473, 'train_steps_per_second': 2.473, 'total_flos': 2090634706944000.0, 'train_loss': 1.3872383524576823, 'epoch': 5.0})

In [26]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

unfrozen_laplacian_model.to(device)

# Test the function with an example from the dataset
text = dataset['test'][1]['text']
label = dataset['test'][1]['label']
print("Predicted label:", predict_label(text,unfrozen_laplacian_model))
print("Expected label:", label)


Predicted label: 0
Expected label: 3


In [27]:
from torch.utils.data import DataLoader

data = dataset_1percent['test']

# Calculate accuracy
accuracy = calculate_accuracy(unfrozen_laplacian_model, data)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.2368


## Alternative: Training a GPT2-Model from Scratch

In [22]:
from transformers import GPT2ForSequenceClassification, GPT2Config

# Define GPT-2 configuration
config = GPT2Config(
    vocab_size=50257,  # Number of tokens in the vocabulary
    n_embd=768,        # Dimensionality of the embeddings and hidden states
    n_layer=12,        # Number of transformer layers
    n_head=12,         # Number of attention heads
    num_labels=4       # Number of labels for sequence classification
)

laplacian_gpt2_model_untrained = GPT2ForSequenceClassification(config)

print(laplacian_gpt2_model_untrained)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


Replace the attention layer with a Laplacian attention layer.

In [23]:
# Replace the attention mechanism in each transformer block
for block in laplacian_gpt2_model_untrained.transformer.h: # accessing each transformer blocks within the GPT-2 model
    block.attn = MultiHeadLaplacianAdaptiveAttentionWrapper(config=laplacian_model.config, num_laplacians=5, norm_axis=1) # and replacing the attention module with the Gaussian attention block.

print(laplacian_gpt2_model_untrained)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadLaplacianAdaptiveAttentionWrapper(
          (attention): MultiHeadLaplacianAdaptiveAttention(
            (attention_heads): ModuleList(
              (0-11): 12 x LaplacianAdaptiveAttention()
            )
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=4, bias=False)
)


Train the Laplacian model from scratch.

In [1]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    # weight_decay=0.01,
    use_cpu = False,
    no_cuda = False
)

trainer = Trainer(
    model=laplacian_gpt2_model_untrained,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

Evaluation

In [ ]:
data = dataset_1percent['test']
accuracy = calculate_accuracy(laplacian_gpt2_model_untrained, data)
print(f"Accuracy: {accuracy:.4f}")